# Webscrape data


In [11]:
import pandas as pd
import os

### Naming convention for the tables

In [12]:
titles_12 = [
    'serie_A_overall',
    'serie_A_homeaway',
    'squad_standard_stats',
    'squad_standard_stats_opp',
    'squad_goalkeeping',
    'squad_goalkeeping_opp',
    'squad_advanced_goalkeeping',
    'squad_advanced_goalkeeping_opp',
    'squad_shooting',
    'squad_shooting_opp',
    'squad_passing',
    'squad_passing_opp',
    'squad_pass_types',
    'squad_pass_types_opp',
    'squad_goal_shot_creation',
    'squad_goal_shot_creation_opp',
    'squad_defensive_actions',
    'squad_defensive_actions_opp',
    'squad_possession',
    'squad_possession_opp',
    'squad_playing_time',
    'squad_playing_time_opp',
    'squad_miscellaneous',
    'squad_miscellaneous_opp',
]

titles_6 = titles_12[:6] + titles_12[8:10] + titles_12[20:]

titles_5 = titles_6[:6] + titles_6[8:]

### Acquire the data

In [13]:
# Fetch all tables from the webpage
os.makedirs('data', exist_ok=True)
for page in range(11):
    
    url = f"https://fbref.com/en/comps/24/{2024 - page}/{2024 - page}-Serie-A-Stats"
    tables = pd.read_html(url)
    year = 2024 - page
    
    # Convert each table to a DataFrame
    if year > 2018:
        dfs = {f"{titles_12[i]}_{year}": table for i, table in enumerate(tables)}
    
    elif year > 2015:
        dfs = {f"{titles_6[i]}_{year}": table for i, table in enumerate(tables)}
    
    else:
        dfs = {f"{titles_5[i]}_{year}": table for i, table in enumerate(tables)}
    
    for name, df in dfs.items():
        df.to_csv(f"data/{name}.csv", index=False)
